In [24]:
import os
from os.path import abspath
import findspark
findspark.init()

In [25]:
os.environ["JAVA_HOME"] = r"/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = r"/home/guilherme/spark-3.4.0-bin-hadoop3"
os.environ["LIVY_CONF_DIR"] = r"/home/guilherme/apache-livy-0.7.1-incubating-bin"

In [26]:
import pyspark.sql.functions as f
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark import SparkContext, SparkConf
import tempfile

warehouse_location = abspath('spark-warehouse')

In [27]:
conf = SparkConf()
conf.set("spark.master","local[*]")
conf.set("spark.submit.deployMode","client")
conf.set("spark.executor.memory", "4g")
conf.set("spark.driver.memory", "4g")
conf.set("spark.yarn.am.memory","1g")
conf.set("spark.eventLog.enabled ","true")
conf.set("spark.sql.adaptive.enabled","true")
spark = SparkSession.builder\
    .config(conf=conf)\
    .config("spark.sql.warehouse.dir", warehouse_location)\
    .config("spark.sql.catalogImplementation", "hive") \
    .enableHiveSupport() \
    .getOrCreate()

23/04/19 05:28:04 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [ ]:
#TutorialBase
#https://docs.cloudera.com/runtime/7.2.10/running-spark-applications/topics/spark-submitting-applications-using-livy.html


#pdf exempl Claudera
#https://docs.cloudera.com/runtime/7.2.10/running-spark-applications/spark-running-applications.pdf

In [28]:
spark

In [29]:
import json
import requests

In [30]:
host = 'http://localhost:8998'
data = {'kind': 'spark'}
headers = {'Content-Type': 'application/json'}
r = requests.post(host + '/sessions', data=json.dumps(data), headers=headers)
r.json()
{u'state': u'starting', u'id': 0, u'kind': u'spark'}


{'state': 'starting', 'id': 0, 'kind': 'spark'}

In [31]:
statement_url = host + r.headers['location']
r = requests.get(statement_url, headers=headers)
print(r.json())
{u'id': 0,
u'output': {u'data': {u'text/plain': u'res0: Long = 10'},
u'execution_count': 0,
u'status': u'ok'},
u'state': u'available'}

{'id': 4, 'name': None, 'appId': None, 'owner': None, 'proxyUser': None, 'state': 'starting', 'kind': 'spark', 'appInfo': {'driverLogUrl': None, 'sparkUiUrl': None}, 'log': ['stdout: ', '\nstderr: ', '23/04/19 05:28:09 WARN Utils: Your hostname, Guilherme resolves to a loopback address: 127.0.1.1; using 172.22.24.52 instead (on interface eth0)', '23/04/19 05:28:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address']}


{'id': 0,
 'output': {'data': {'text/plain': 'res0: Long = 10'},
  'execution_count': 0,
  'status': 'ok'},
 'state': 'available'}

In [23]:
#POST /sessions/{sessionId}/statements
data = {'code': 'sc.parallelize(1 to 10).count()'}
r = requests.post(statement_url, data=json.dumps(data), headers=headers)

r.json()
{u'output': None, u'state': u'running', u'id': 0}



JSONDecodeError: Expecting value: line 1 column 1 (char 0)

{'id': 2, 'name': None, 'appId': None, 'owner': None, 'proxyUser': None, 'state': 'dead', 'kind': 'spark', 'appInfo': {'driverLogUrl': None, 'sparkUiUrl': None}, 'log': ['\tat org.apache.spark.deploy.SparkSubmit.submit(SparkSubmit.scala:215)', '\tat org.apache.spark.deploy.SparkSubmit.doSubmit(SparkSubmit.scala:91)', '\tat org.apache.spark.deploy.SparkSubmit$$anon$2.doSubmit(SparkSubmit.scala:1111)', '\tat org.apache.spark.deploy.SparkSubmit$.main(SparkSubmit.scala:1120)', '\tat org.apache.spark.deploy.SparkSubmit.main(SparkSubmit.scala)', 'Caused by: java.lang.ClassNotFoundException: scala.Function0$class', '\tat java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:476)', '\tat java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:589)', '\tat java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:522)', '\t... 20 more']}


{'id': 0,
 'output': {'data': {'text/plain': 'res0: Long = 10'},
  'execution_count': 0,
  'status': 'ok'},
 'state': 'available'}